# Clasificación de Especies de Iris con Redes Neuronales (PyTorch) y Regularización

**Disciplina:** Aprendizaje Profundo, Redes Neuronales, Clasificación, PyTorch

**Objetivo:**
El objetivo de este notebook es construir, entrenar y evaluar una red neuronal para clasificar las especies de flores del dataset Iris utilizando PyTorch. Se incorporarán técnicas de preprocesamiento, regularización (Weight Decay y Dropout) y manejo explícito del bucle de entrenamiento con funcionalidades equivalentes a callbacks (EarlyStopping, ModelCheckpoint, ReduceLROnPlateau) para mejorar el entrenamiento y la robustez del modelo.

## 1. Carga de Librerías y Configuración Inicial

**Propósito de esta sección:**
Importar todas las bibliotecas necesarias y configurar el entorno para el análisis, incluyendo la fijación de semillas para reproducibilidad y la configuración del dispositivo (CPU/GPU).

**Bibliotecas Clave:**
* **`numpy`, `pandas`**: Para manipulación de datos.
* **`matplotlib.pyplot`, `seaborn`**: Para visualizaciones.
* **`sklearn.datasets`**: Para cargar el dataset Iris.
* **`sklearn.model_selection`**: Para `train_test_split`.
* **`sklearn.preprocessing`**: Para `StandardScaler`.
* **`sklearn.metrics`**: Para `classification_report` y `confusion_matrix`.
* **`torch`, `torch.nn`, `torch.optim`, `torch.utils.data`**: Para construir y entrenar la red neuronal con PyTorch.

In [ ]:
# Comandos mágicos de IPython (opcional en scripts)
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# Importación de bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import copy # Para model checkpointing

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Importar PyTorch
PYTORCH_IMPORTED_SUCCESSFULLY = False # <--- DEFINIR LA BANDERA AQUÍ
try:
    import torch
    import torch.nn as nn
    import torch.optim as optim
    from torch.utils.data import TensorDataset, DataLoader
    print(f"Biblioteca 'torch' importada correctamente. Versión: {torch.__version__}")
    PYTORCH_IMPORTED_SUCCESSFULLY = True # <--- ACTUALIZAR LA BANDERA
except ImportError as e:
    print(f"Error al importar 'torch': {e}")
    print("Por favor, instálala con 'pip install torch torchvision torchaudio' (o según tu sistema).")
    print("El script continuará, pero las secciones de PyTorch probablemente fallarán.")


# Configuración para reproducibilidad
SEED = 42
np.random.seed(SEED)
if PYTORCH_IMPORTED_SUCCESSFULLY: # Solo si PyTorch se importó
    torch.manual_seed(SEED)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(SEED)

# Configuración de dispositivo (GPU si está disponible, sino CPU)
device = torch.device("cuda" if PYTORCH_IMPORTED_SUCCESSFULLY and torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

# Configuración de estilo y visualización
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 10

## 2. Funciones Personalizadas

### Descripción de la Función: `cargar_y_preparar_datos_iris_pytorch`

**Objetivo Principal:**
Cargar el dataset Iris, realizar preprocesamiento (escalado), convertir a tensores de PyTorch y crear DataLoaders.

**Características:**
* **Procesamiento:**
    1. Carga el dataset Iris.
    2. Separa características (X) y objetivo (y).
    3. Escala las características X usando `StandardScaler`.
    4. Convierte X e y a tensores de PyTorch. El objetivo `y` se convierte a `torch.long` para `CrossEntropyLoss`.
    5. Divide los datos en conjuntos de entrenamiento, validación y prueba.
    6. Crea `TensorDataset` y `DataLoader` para cada conjunto.
* **Valor de Retorno:**
    * `train_loader, val_loader, test_loader`: DataLoaders para PyTorch.
    * `scaler`: El objeto `StandardScaler` ajustado.
    * `feature_names`, `target_names`: Nombres.
    * `X_train_np, X_val_np, X_test_np, y_train_np, y_val_np, y_test_np`: Datos NumPy para referencia.

In [ ]:
def cargar_y_preparar_datos_iris_pytorch(batch_size=16, test_size=0.2, val_size=0.1, random_state=SEED):
    """
    Carga, preprocesa el dataset Iris, lo convierte a tensores PyTorch y crea DataLoaders.
    """
    print("Cargando y preparando el dataset Iris para PyTorch...")
    iris = load_iris()
    X_np = iris.data
    y_np = iris.target 
    feature_names = iris.feature_names
    target_names = iris.target_names

    scaler = StandardScaler()
    X_scaled_np = scaler.fit_transform(X_np)

    X_temp_np, X_test_np, y_temp_np, y_test_np = train_test_split(
        X_scaled_np, y_np, test_size=test_size, random_state=random_state, stratify=y_np
    )
    
    val_proportion_of_temp = val_size / (1 - test_size) if (1-test_size) > 0 else 0
    if val_proportion_of_temp == 0 and len(X_temp_np)>0 : # Si no hay val_size o temp es muy chico
        X_train_np, X_val_np, y_train_np, y_val_np = X_temp_np, np.array([]), y_temp_np, np.array([])
        # Crear X_val_np, y_val_np vacíos pero con la forma correcta para evitar errores posteriores
        X_val_np = np.empty((0, X_train_np.shape[1]), dtype=X_train_np.dtype)
        y_val_np = np.empty((0,), dtype=y_train_np.dtype)

    elif val_proportion_of_temp > 0 :
         X_train_np, X_val_np, y_train_np, y_val_np = train_test_split(
            X_temp_np, y_temp_np, test_size=val_proportion_of_temp, random_state=random_state, stratify=y_temp_np
        )
    else: # Caso X_temp_np está vacío
        X_train_np, X_val_np, y_train_np, y_val_np = X_temp_np, X_temp_np, y_temp_np, y_temp_np


    # Convertir a tensores PyTorch solo si PyTorch se importó
    if PYTORCH_IMPORTED_SUCCESSFULLY:
        X_train_torch = torch.tensor(X_train_np, dtype=torch.float32)
        y_train_torch = torch.tensor(y_train_np, dtype=torch.long)
        X_val_torch = torch.tensor(X_val_np, dtype=torch.float32)
        y_val_torch = torch.tensor(y_val_np, dtype=torch.long)
        X_test_torch = torch.tensor(X_test_np, dtype=torch.float32)
        y_test_torch = torch.tensor(y_test_np, dtype=torch.long)

        train_dataset = TensorDataset(X_train_torch, y_train_torch)
        # Solo crear val_dataset/loader si X_val_torch no está vacío
        val_loader = None
        if X_val_torch.shape[0] > 0:
            val_dataset = TensorDataset(X_val_torch, y_val_torch)
            val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
        
        test_dataset = TensorDataset(X_test_torch, y_test_torch)

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    else:
        train_loader, val_loader, test_loader = None, None, None

    print(f"\nDimensiones NumPy: X_train: {X_train_np.shape}, y_train: {y_train_np.shape}")
    print(f"Dimensiones NumPy: X_val: {X_val_np.shape}, y_val: {y_val_np.shape}")
    print(f"Dimensiones NumPy: X_test: {X_test_np.shape}, y_test: {y_test_np.shape}")
    if train_loader:
        print(f"Tamaños DataLoaders: Train: {len(train_loader.dataset)}, Val: {len(val_loader.dataset) if val_loader else 0}, Test: {len(test_loader.dataset)}")
    
    return (train_loader, val_loader, test_loader, 
            scaler, feature_names, target_names, 
            X_train_np, X_val_np, X_test_np, # Devolver también X_train y X_val en NumPy
            y_train_np, y_val_np, y_test_np)

### Descripción de la Clase: `IrisClassifierNet`
(Sin cambios)

In [ ]:
if PYTORCH_IMPORTED_SUCCESSFULLY:
    class IrisClassifierNet(nn.Module):
        def __init__(self, input_dim, output_dim, dropout_rate=0.3):
            super(IrisClassifierNet, self).__init__()
            self.fc1 = nn.Linear(input_dim, 64)
            self.relu1 = nn.ReLU()
            self.dropout1 = nn.Dropout(dropout_rate)
            
            self.fc2 = nn.Linear(64, 32)
            self.relu2 = nn.ReLU()
            self.dropout2 = nn.Dropout(dropout_rate)
            
            self.fc3 = nn.Linear(32, output_dim)

        def forward(self, x):
            x = self.fc1(x)
            x = self.relu1(x)
            x = self.dropout1(x)
            
            x = self.fc2(x)
            x = self.relu2(x)
            x = self.dropout2(x)
            
            x = self.fc3(x) 
            return x
else:
    IrisClassifierNet = None # Definir como None si PyTorch no está
    print("PyTorch no importado, IrisClassifierNet no definido.")

### Descripción de la Función: `entrenar_modelo_pytorch`
(Sin cambios, pero su ejecución dependerá de PYTORCH_IMPORTED_SUCCESSFULLY)

In [ ]:
def entrenar_modelo_pytorch(model, train_loader, val_loader, criterion, optimizer, scheduler, 
                            num_epochs, patience, model_save_path, device):
    if not PYTORCH_IMPORTED_SUCCESSFULLY or model is None:
        print("PyTorch no disponible o modelo no definido. Saltando entrenamiento.")
        return {"train_loss": [], "val_loss": [], "train_acc": [], "val_acc": []} # Devolver historial vacío

    print(f"\nIniciando entrenamiento en {device} por {num_epochs} épocas...")
    
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []
    
    best_val_loss = float('inf')
    epochs_no_improve = 0
    best_model_state = None

    for epoch in range(num_epochs):
        model.train()
        running_loss_train = 0.0
        correct_train = 0
        total_train = 0
        
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss_train += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()
            
        epoch_loss_train = running_loss_train / total_train if total_train > 0 else 0
        epoch_acc_train = correct_train / total_train if total_train > 0 else 0
        train_losses.append(epoch_loss_train)
        train_accuracies.append(epoch_acc_train)

        # Fase de Validación (solo si val_loader existe y tiene datos)
        epoch_loss_val = float('nan') # Valor por defecto si no hay validación
        epoch_acc_val = float('nan')
        
        if val_loader and len(val_loader.dataset) > 0:
            model.eval()
            running_loss_val = 0.0
            correct_val = 0
            total_val = 0
            
            with torch.no_grad():
                for inputs, labels in val_loader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = model(inputs)
                    loss_val_batch = criterion(outputs, labels)
                    
                    running_loss_val += loss_val_batch.item() * inputs.size(0)
                    _, predicted_val = torch.max(outputs.data, 1)
                    total_val += labels.size(0)
                    correct_val += (predicted_val == labels).sum().item()
            
            epoch_loss_val = running_loss_val / total_val if total_val > 0 else 0
            epoch_acc_val = correct_val / total_val if total_val > 0 else 0
        
        val_losses.append(epoch_loss_val)
        val_accuracies.append(epoch_acc_val)
        
        print(f"Época {epoch+1}/{num_epochs} | "
              f"Pérdida Ent.: {epoch_loss_train:.4f} | Acc Ent.: {epoch_acc_train:.4f} | "
              f"Pérdida Val.: {epoch_loss_val:.4f} | Acc Val.: {epoch_acc_val:.4f}")

        if scheduler and not np.isnan(epoch_loss_val): # Solo hacer step si hay val_loss
            scheduler.step(epoch_loss_val)

        if not np.isnan(epoch_loss_val) and epoch_loss_val < best_val_loss:
            best_val_loss = epoch_loss_val
            best_model_state = copy.deepcopy(model.state_dict())
            torch.save(best_model_state, model_save_path)
            print(f"  Mejora en validación (Pérdida Val.: {best_val_loss:.4f}). Guardando modelo en {model_save_path}")
            epochs_no_improve = 0
        elif not np.isnan(epoch_loss_val): # Si hay val_loss pero no mejoró
            epochs_no_improve += 1
            print(f"  Sin mejora en validación por {epochs_no_improve} épocas.")
            if epochs_no_improve >= patience:
                print(f"Early stopping activado en la época {epoch+1}. Mejor val_loss: {best_val_loss:.4f}")
                break
        elif np.isnan(epoch_loss_val) and epoch == 0: # Si no hay val_loader, guardamos el primer modelo
             best_model_state = copy.deepcopy(model.state_dict())
             torch.save(best_model_state, model_save_path)
             print(f"  No hay set de validación. Guardando modelo de época 1 en {model_save_path}")


    if best_model_state:
        print(f"Cargando los pesos del mejor modelo desde {model_save_path} (mejor val_loss: {best_val_loss:.4f})")
        model.load_state_dict(best_model_state)
        
    return {"train_loss": train_losses, "val_loss": val_losses, 
            "train_acc": train_accuracies, "val_acc": val_accuracies}

### Descripción de la Función: `graficar_historial_pytorch`
(Sin cambios)

In [ ]:
def graficar_historial_pytorch(history_dict):
    print("\nGraficando historial de entrenamiento (PyTorch)...")
    train_acc = history_dict.get('train_acc')
    val_acc = history_dict.get('val_acc')
    train_loss = history_dict.get('train_loss')
    val_loss = history_dict.get('val_loss')
    
    # Filtrar NaNs de las listas de validación si no hubo val_loader
    val_acc_plot = [v for v in val_acc if not np.isnan(v)] if val_acc else []
    val_loss_plot = [v for v in val_loss if not np.isnan(v)] if val_loss else []
    
    # Ajustar epochs_range para que coincida con la longitud de los datos de entrenamiento,
    # y las validaciones solo se plotearán hasta donde tengan datos.
    epochs_range_train = range(len(train_loss if train_loss else train_acc))
    epochs_range_val = range(len(val_loss_plot if val_loss_plot else val_acc_plot))


    plt.figure(figsize=(14, 5))

    if train_acc: # Siempre graficar entrenamiento
        plt.subplot(1, 2, 1)
        plt.plot(epochs_range_train, train_acc, label='Precisión (Entrenamiento)')
        if val_acc_plot: # Solo graficar validación si hay datos
            plt.plot(epochs_range_val, val_acc_plot, label='Precisión (Validación)')
        plt.legend(loc='lower right')
        plt.title('Precisión de Entrenamiento y Validación')
        plt.xlabel('Épocas'); plt.ylabel('Precisión')

    if train_loss: # Siempre graficar entrenamiento
        plt.subplot(1, 2, 2)
        plt.plot(epochs_range_train, train_loss, label='Pérdida (Entrenamiento)')
        if val_loss_plot: # Solo graficar validación si hay datos
            plt.plot(epochs_range_val, val_loss_plot, label='Pérdida (Validación)')
        plt.legend(loc='upper right')
        plt.title('Pérdida de Entrenamiento y Validación')
        plt.xlabel('Épocas'); plt.ylabel('Pérdida')
    plt.show()

### Descripción de la Función: `evaluar_modelo_clasificacion_pytorch`
(Sin cambios)

In [ ]:
def evaluar_modelo_clasificacion_pytorch(model, test_loader, criterion, target_names, device, X_test_numpy_orig, y_test_numpy_orig):
    if not PYTORCH_IMPORTED_SUCCESSFULLY or model is None:
        print("PyTorch no disponible o modelo no definido. Saltando evaluación.")
        return

    print("\nEvaluando el modelo PyTorch en el conjunto de prueba...")
    model.eval()
    test_loss = 0.0
    all_preds_np = []
    all_labels_np = [] # No usado para métricas finales si se usa y_test_numpy_orig
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            
            _, predicted = torch.max(outputs.data, 1)
            all_preds_np.extend(predicted.cpu().numpy())
            # all_labels_np.extend(labels.cpu().numpy()) # No es necesario si usamos y_test_numpy_orig
            
    avg_test_loss = test_loss / len(y_test_numpy_orig) if len(y_test_numpy_orig) > 0 else 0
    # Usar y_test_numpy_orig que es la lista completa de etiquetas verdaderas del test set
    final_preds_np = np.array(all_preds_np)
    test_accuracy_sklearn = accuracy_score(y_test_numpy_orig, final_preds_np)

    print(f"Pérdida Promedio en el conjunto de prueba: {avg_test_loss:.4f}")
    print(f"Precisión (sklearn) en el conjunto de prueba: {test_accuracy_sklearn:.4f}")

    print("\nReporte de Clasificación (PyTorch):")
    print(classification_report(y_test_numpy_orig, final_preds_np, target_names=target_names, zero_division=0))

    cm = confusion_matrix(y_test_numpy_orig, final_preds_np)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=target_names, yticklabels=target_names)
    plt.title('Matriz de Confusión (PyTorch)')
    plt.xlabel('Predicción'); plt.ylabel('Real')
    plt.show()

## 3. Desarrollo del Ejercicio: Clasificación de Iris con PyTorch

### 3.1. Carga y Preparación de Datos

In [ ]:
BATCH_SIZE_IRIS = 8
# Ajustar val_size a 0.2 para que sea una proporción del 80% restante si test_size=0.2
# val_size_proportion = 0.2 / 0.8 = 0.25
(train_loader_iris, val_loader_iris, test_loader_iris, 
 scaler_iris, iris_feature_names, iris_target_names,
 X_train_iris_np, X_val_iris_np, X_test_iris_np,
 y_train_iris_np, y_val_iris_np, y_test_iris_np) = cargar_y_preparar_datos_iris_pytorch(
                                                        batch_size=BATCH_SIZE_IRIS, 
                                                        test_size=0.2, 
                                                        val_size=0.15 # ej: 15% del original para validación
                                                    )

### 3.2. Creación del Modelo de Clasificación

In [ ]:
modelo_iris_pytorch = None # Inicializar
if PYTORCH_IMPORTED_SUCCESSFULLY and IrisClassifierNet is not None:
    # input_dim_iris_actual = X_train_iris_np.shape[1] # Usar el X_train_np devuelto
    if X_train_iris_np.shape[0] > 0: # Asegurarse que X_train_np no está vacío
         input_dim_iris_actual = X_train_iris_np.shape[1]
    elif train_loader_iris and len(train_loader_iris.dataset)>0: #Fallback al loader si X_train_np no se puede usar
        sample_inputs, _ = next(iter(train_loader_iris))
        input_dim_iris_actual = sample_inputs.shape[1]
    else:
        print("No se pudo determinar input_dim para el modelo Iris. Usando valor por defecto 4.")
        input_dim_iris_actual = 4 # Valor por defecto para Iris
        
    output_dim_iris = len(iris_target_names) 

    modelo_iris_pytorch = IrisClassifierNet(input_dim_iris_actual, output_dim_iris, dropout_rate=0.25).to(device)
    print("\nResumen del modelo PyTorch (estructura):")
    print(modelo_iris_pytorch)
else:
    print("PyTorch no se importó o IrisClassifierNet no definido, saltando creación de modelo.")

### 3.3. Definición de Pérdida, Optimizador y Scheduler

In [ ]:
criterion_iris, optimizer_iris, scheduler_iris = None, None, None # Inicializar
if PYTORCH_IMPORTED_SUCCESSFULLY and modelo_iris_pytorch:
    criterion_iris = nn.CrossEntropyLoss()
    optimizer_iris = optim.Adam(modelo_iris_pytorch.parameters(), lr=0.001, weight_decay=0.0005) 
    scheduler_iris = optim.lr_scheduler.ReduceLROnPlateau(optimizer_iris, 'min', patience=7, factor=0.2, verbose=True, min_lr=1e-6)
else:
    print("Modelo no definido, no se pueden crear criterio/optimizador.")

### 3.4. Entrenamiento del Modelo

In [ ]:
NUM_EPOCHS_IRIS = 150 # Reducido de 200 para ejecución más rápida
PATIENCE_IRIS = 15
MODEL_SAVE_PATH_IRIS = 'best_iris_model_pytorch.pth'
history_iris_pytorch = None

if modelo_iris_pytorch and criterion_iris and optimizer_iris and train_loader_iris and val_loader_iris:
    history_iris_pytorch = entrenar_modelo_pytorch(
        modelo_iris_pytorch, train_loader_iris, val_loader_iris,
        criterion_iris, optimizer_iris, scheduler_iris,
        num_epochs=NUM_EPOCHS_IRIS,
        patience=PATIENCE_IRIS,
        model_save_path=MODEL_SAVE_PATH_IRIS,
        device=device
    )
else:
    print("Componentes de entrenamiento no disponibles. Saltando entrenamiento.")

### 3.5. Visualización del Historial de Entrenamiento

In [ ]:
if history_iris_pytorch:
    graficar_historial_pytorch(history_iris_pytorch)

### 3.6. Evaluación del Modelo
El mejor modelo ya fue cargado al final de la función `entrenar_modelo_pytorch`.

In [ ]:
if modelo_iris_pytorch and history_iris_pytorch and criterion_iris and test_loader_iris:
    evaluar_modelo_clasificacion_pytorch(
        modelo_iris_pytorch, test_loader_iris, criterion_iris, 
        iris_target_names, device,
        X_test_iris_np, y_test_iris_np 
    )
else:
    print("Modelo no entrenado o componentes no disponibles. Saltando evaluación.")

## 4. Conclusiones del Ejercicio (Clasificación Iris con PyTorch)

**Resumen de Hallazgos:**
* Se cargó y preprocesó el dataset Iris, escalando características y convirtiendo los datos a tensores de PyTorch para su uso con DataLoaders. Se creó un conjunto de validación.
* Se definió una red neuronal (`IrisClassifierNet`) usando `torch.nn.Module`, con capas lineales, activaciones ReLU y Dropout.
* El modelo fue compilado con `CrossEntropyLoss` y el optimizador `Adam` (con `weight_decay` para regularización L2).
* Se implementó un bucle de entrenamiento explícito, incorporando:
    * Validación en cada época.
    * `ReduceLROnPlateau` para ajustar dinámicamente la tasa de aprendizaje.
    * Lógica manual para `EarlyStopping` y `ModelCheckpointing`.
* La precisión final alcanzada en el conjunto de prueba fue de **[Completar con la precisión obtenida]**.
* El reporte de clasificación y la matriz de confusión mostraron el rendimiento del modelo en cada clase.
* Las curvas de aprendizaje indicaron **[Describir si hubo sobreajuste, si EarlyStopping actuó, etc.]**.

**Aprendizaje General:**
Este ejercicio demostró cómo construir, entrenar y evaluar una red neuronal para clasificación multiclase utilizando PyTorch. Se destacó la flexibilidad y el control que ofrece PyTorch al requerir la escritura explícita del bucle de entrenamiento y la gestión de callbacks. Se aplicaron técnicas de regularización como Dropout y decaimiento de pesos (L2) para mejorar la generalización del modelo.

*(Nota: Los resultados específicos deben completarse después de ejecutar completamente el notebook.)*